In [1]:
import sys
sys.path.append("../../")
from taxcalc import *
import pandas as pd
import numpy as np
import copy

In [2]:
CBO_growth = {2013: {'_factor_target': [0.022, 
                                        0.024,
                                        0.024,
                                        0.025,
                                        0.026,
                                        0.023,
                                        0.018,
                                        0.019,
                                        0.021,
                                        0.021,
                                        0.021,
                                        0.02,
                                        0.02,
                                        0.02]
                     }
              }
behavior_traits = {2017: {'_BE_inc': [0.0], '_BE_sub': [0.4], '_BE_cg': [-3.487]}}

# Create the policies and calculators

In [3]:
policy_cl = Policy()
behavior_cl = Behavior()
records_cl = Records("../../puf.csv")
calc_cl = Calculator(policy_cl, records_cl, behavior_traits)
calc_cl.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_cl.increment_year()
assert calc_cl.current_year == 2017
calc_cl.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [4]:
policy_amt = Policy()
behavior_amt = Behavior()
records_amt = Records("../../puf.csv")
reform_amt = {
    2017: {
        '_AMT_trt1': [0.0],
        '_AMT_trt2': [0.0]
    }
}
policy_amt.implement_reform(reform_amt)
calc_amt = Calculator(policy_amt, records_amt, behavior_amt)
calc_amt.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_amt.increment_year()
assert calc_amt.current_year == 2017
calc_amt.calc_all()
calc_amt.behavior.update_behavior(behavior_traits)
calc_amt_beh = Behavior.response(calc_cl, calc_amt)
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [4]:
policy_bush = Policy()
behavior_bush = Behavior()
records_bush = Records("../../puf.csv")
reform_bush = {
    2017: {
        '_II_rt7': [0.35],
        '_CG_rt3': [0.15],
        '_II_prt': [0.0],
        '_ID_prt': [0.0],
        '_AMT_CG_rt3': [0.15],
        '_NIIT_trt': [0.0],
        '_PT_rt7': [0.35]
    }
}
policy_bush.implement_reform(reform_bush)
calc_bush = Calculator(policy_bush, records_bush, behavior_bush)
calc_bush.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_bush.increment_year()
assert calc_bush.current_year == 2017
calc_bush.calc_all()
calc_bush.behavior.update_behavior(behavior_traits)
calc_bush_beh = Behavior.response(calc_cl, calc_bush)
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [5]:
policy_all = Policy()
behavior_all = Behavior()
records_all = Records("../../puf.csv")
reform_all = {
    2017: {
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt3': [0.25],
        '_II_rt4': [0.25],
        '_II_rt5': [0.33],
        '_II_rt6': [0.33],
        '_II_rt7': [0.33],
        '_AMT_trt1': [0.0],
        '_AMT_trt2': [0.0],
        '_STD': [[12000, 24000, 12000, 18000, 24000, 12000, 1050]],
        '_STD_Aged': [[0,0,0,0,0,0],
                      [0,0,0,0,0,0],
                      [0,0,0,0,0,0],
                      [0,0,0,0,0,0]],
        '_II_em': [0,0,0,0],
        '_CTC_ps': [[75000, 150000, 75000, 75000, 75000, 75000]],
        '_II_credit': [[500, 500, 500, 500, 500, 500]],
        '_II_credit_ps': [[75000, 150000, 75000, 75000, 75000, 75000]],
        '_II_credit_prt': [0.05],
        '_ID_StateLocalTax_HC': [1.0],
        '_ID_Medical_HC': [1.0],
        '_ID_Casualty_HC': [1.0],
        '_ID_Miscellaneous_HC': [1.0],
        '_ID_RealEstate_HC': [1.0],
        '_ALD_Investment_ec': [0.5],        
        '_CG_as_II': [1],
        '_NIIT_trt': [0.0],
        '_PT_rt1': [0.12],
        '_PT_rt2': [0.12],
        '_PT_rt3': [0.25],
        '_PT_rt4': [0.25],
        '_PT_rt5': [0.25],
        '_PT_rt6': [0.25],
        '_PT_rt7': [0.25],
    }
}
policy_all.implement_reform(reform_all)
calc_all = Calculator(policy_all, records_all, behavior_all)
calc_all.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_all.increment_year()
assert calc_all.current_year == 2017
calc_all.calc_all()
calc_all.behavior.update_behavior(behavior_traits)
calc_all_beh = Behavior.response(calc_cl, calc_all)
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


# Iterator and revenue impact

In [7]:
def static_path_change(calcx, calcy):
    static_path = [0] * 10
    calc1 = copy.deepcopy(calcx)
    calc2 = copy.deepcopy(calcy)
    for i in range(10):
        calc1.calc_all()
        calc2.calc_all()
        calc1_combined = (calc1.records._combined * calc1.records.s006)
        calc2_combined = (calc2.records._combined * calc2.records.s006)
        static_path[i] = (calc2_combined - calc1_combined).sum() / 10**9
        if calc1.current_year < 2026:
            calc1.increment_year()
            calc2.increment_year()
    return static_path

In [8]:
path1 = static_path_change(calc_cl, calc_all)
print 'Path assuming no behavioral response'
for x in path1:
    print x

Path assuming no behavioral response
-97.9137941446
-97.4789735132
-95.9056333998
-95.7266429503
-96.6918511548
-98.4224193285
-100.901734911
-104.326659845
-108.675065937
-114.135751242


In [9]:
path2 = static_path_change(calc_cl, calc_all_beh)
print 'Path with behavioral response'
for x in path2:
    print x

Path with behavioral response
-33.7856774295
-30.7073640824
-26.8919947083
-23.8271587252
-21.3541356098
-19.4740586896
-18.0121151182
-17.3429501593
-17.3649888279
-18.5453311168


In [10]:
path3 = static_path_change(calc_amt, calc_all)
print 'Path assuming no behavioral response, compared against just AMT repeal'
for x in path3:
    print x

Path assuming no behavioral response, compared against just AMT repeal
-56.1204170172
-52.9642491727
-48.7411504756
-45.6313003599
-43.5347063674
-41.8481147726
-40.8672107143
-40.8007624044
-41.5313859206
-43.2507739781


In [11]:
path4 = static_path_change(calc_amt_beh, calc_all_beh)
print 'Path with behavioral response, compared against just AMT repeal'
for x in path4:
    print x

Path with behavioral response, compared against just AMT repeal
6.35064684441
12.0796494452
18.4936510697
24.4347534074
29.8262695664
34.9941775053
39.7173326136
43.6128074154
46.9728568591
49.2545132272


In [12]:
path5 = static_path_change(calc_bush, calc_all)
print 'Path assuming no behavioral response, compared against pre-2009 law'
for x in path5:
    print x

Path assuming no behavioral response, compared against pre-2009 law
-2.46868470222
0.937239267559
4.7353560907
8.35560934396
11.7131167702
14.8701680779
17.9664226125
20.5228205109
22.8844044817
24.6718040024


In [13]:
path6 = static_path_change(calc_bush_beh, calc_all_beh)
print 'Path with behavioral response, compared against pre-2009 law'
for x in path6:
    print x

Path with behavioral response, compared against pre-2009 law
2.39478104214
6.54697262585
10.9481708385
15.3375947198
19.4994965726
23.2869785203
27.2401935933
30.6202144182
33.7535017226
36.2034773136


# Diagnostics

In [25]:
def taxableGain(calc):
    tg = np.maximum(0., np.minimum(calc.records.c23650, calc.records.p23250)) + calc.records.e01100
    return tg

def num_ided(calc):
    ided = calc.records.s006[(calc.records.c04470 > 0.) * (calc.records.c00100 > 0.)].sum()
    return ided
def num_std(calc):
    std = calc.records.s006[(calc.records._standard > 0.) * (calc.records.c00100 > 0.)].sum()
    return std
def wavgMTR_wages(calc):
    mtr = ((calc.mtr('e00200p')[1] * calc.records.e00200 * calc.records.s006).sum() / 
                   (calc.records.e00200 * calc.records.s006).sum())
    return mtr
def wavgMTR_cg(calc):
    gain = taxableGain(calc)
    mtr = ((calc.mtr('p23250')[1] * gain * calc.records.s006).sum() / 
                 (gain * calc.records.s006).sum())
    return mtr
def wavgMTR_int(calc):
    mtr = ((calc.mtr('e00300')[1] * calc.records.c00300 * calc.records.s006).sum() / 
                   (calc.records.c00300 * calc.records.s006).sum())
    return mtr
def wavgMTR_div(calc):
    mtr = ((calc.mtr('e00650')[1] * calc.records.c00650 * calc.records.s006).sum() / 
                   (calc.records.c00650 * calc.records.s006).sum())
    return mtr
def wavgMTR_stcg(calc):
    gain = taxableGain(calc)
    mtr = ((calc.mtr('p22250')[1] * gain * calc.records.s006).sum() / 
                 (gain * calc.records.s006).sum())
    return mtr
def wavgMTR_passthru(calc):
    passthruInc = abs(calc.records.c00900) + abs(calc.records.c26270)
    mtr_c = calc.mtr('e00900p')[1]
    mtr_e = calc.mtr('e02000')[1]    
    wtot = mtr_c * abs(calc.records.c00900) + mtr_e * abs(calc.records.c26270)
    mtr = (wtot * calc.records.s006).sum() / (passthruInc * calc.records.s006).sum()
    return mtr
def wavgMTR_sp(calc):
    sp_inc = abs(calc.records.c00900)
    mtr = ((calc.mtr('e00900p')[1] * sp_inc * calc.records.s006).sum() / 
           (sp_inc * calc.records.s006).sum())
    return mtr
def wavgMTR_schE(calc):
    schE_inc = abs(calc.records.c26270)
    mtr = ((calc.mtr('e02000')[1] * schE_inc * calc.records.s006).sum() / 
                   (schE_inc * calc.records.s006).sum())
    return mtr
def no_inc_tax(calc):
    count = calc.records.s006[calc.records._iitax < 0.0001].sum()
    return count

def lwrMTR_wages(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('e00200p')[2] < calc1.mtr('e00200p')[2])].sum()
    return diff
def lwrMTR_cg(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('p23250')[2] < calc1.mtr('p23250')[2])].sum()
    return diff
def hgrMTR_wages(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('e00200p')[2] > calc1.mtr('e00200p')[2])].sum()
    return diff
def hgrMTR_cg(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[(calc2.mtr('p23250')[2] > calc1.mtr('p23250')[2])].sum()
    return diff
def num_taxcut(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    diff = calc2.records.s006[calc1.records._combined > calc2.records._combined + 0.0001].sum()
    return diff

def diagnose1(calc):
    print str(round(num_ided(calc) / 1000) / 1000.0) + " million taxpayers using itemized deductions"
    print str(round(num_std(calc) / 1000) / 1000.0) + " million taxpayers using the standard deduction"
    print "Average MTR on wage income: " + str(round(wavgMTR_wages(calc) * 10000) / 10000.0)
    print "Average MTR on long-term gains: " + str(round(wavgMTR_cg(calc) * 10000) / 10000.0)
    print "Average MTR on short-term gains: " + str(round(wavgMTR_stcg(calc) * 10000) / 10000.0)
    print "Average MTR on interest: " + str(round(wavgMTR_int(calc) * 10000) / 10000.0)
    print "Average MTR on qualified dividends: " + str(round(wavgMTR_div(calc) * 10000) / 10000.0)
    print "Average MTR on pass-through income: " + str(round(wavgMTR_passthru(calc) * 10000) / 10000.0)
    print "Average MTR on sole proprietorships: " + str(round(wavgMTR_sp(calc) * 10000) / 10000.0)
    print "Average MTR on partnership/S-corp: " + str(round(wavgMTR_schE(calc) * 10000) / 10000.0)
    print "Number paying no income tax: " + str(round(no_inc_tax(calc) / 1000) / 1000.0) + " million"
    
def diagnose2(calc1, calc2):
    #calc1 is baseline, calc2 is reform
    print "Number with lower EMTR on wages: " + str(round(lwrMTR_wages(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number with higher EMTR on wages: " + str(round(hgrMTR_wages(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number with lower EMTR on gains: " + str(round(lwrMTR_cg(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number with higher EMTR on gains: " + str(round(hgrMTR_cg(calc1, calc2) / 1000) / 1000.0) + " million"
    print "Number receiving a tax cut: " + str(round(num_taxcut(calc1, calc2) / 1000) / 1000.0) + " million"

In [26]:
diagnose1(calc_cl)

45.42 million taxpayers using itemized deductions
113.603 million taxpayers using the standard deduction
Average MTR on wage income: 0.2365
Average MTR on long-term gains: 0.222
Average MTR on short-term gains: 0.3136
Average MTR on interest: 0.2378
Average MTR on qualified dividends: 0.1756
Average MTR on pass-through income: 0.2955
Average MTR on sole proprietorships: 0.2771
Average MTR on partnership/S-corp: 0.3061
Number paying no income tax: 66.972 million


In [7]:
diagnose1(calc_all)

8.539 million taxpayers using itemized deductions
150.262 million taxpayers using the standard deduction
Average MTR on wage income: 0.2189
Average MTR on long-term gains: 0.1501
Average MTR on short-term gains: 0.3002
Average MTR on interest: 0.0982
Average MTR on qualified dividends: 0.1254
Average MTR on pass-through income: 0.2274
Average MTR on sole proprietorships: 0.2497
Average MTR on partnership/S-corp: 0.2147
Number paying no income tax: 61.042 million


In [8]:
diagnose1(calc_all_beh)

8.432 million taxpayers using itemized deductions
150.372 million taxpayers using the standard deduction
Average MTR on wage income: 0.2211
Average MTR on long-term gains: 0.1555
Average MTR on short-term gains: 0.311
Average MTR on interest: 0.1171
Average MTR on qualified dividends: 0.1268
Average MTR on pass-through income: 0.2289
Average MTR on sole proprietorships: 0.2502
Average MTR on partnership/S-corp: 0.2167
Number paying no income tax: 60.757 million


In [18]:
diagnose2(calc_cl, calc_all)

Number with lower EMTR on wages: 70.861 million
Number with higher EMTR on wages: 36.092 million
Number with lower EMTR on gains: 52.959 million
Number with higher EMTR on gains: 55.786 million
Number receiving a tax cut: 66.983 million


In [19]:
diagnose2(calc_cl, calc_all_beh)

Number with lower EMTR on wages: 69.293 million
Number with higher EMTR on wages: 34.302 million
Number with lower EMTR on gains: 52.996 million
Number with higher EMTR on gains: 55.865 million
Number receiving a tax cut: 62.508 million


# Average tax rates

In [15]:
def getAGI(calc1):
    return np.asarray(np.where(calc1.records.c00100 < 0, 0., calc1.records.c00100))

def getAvgRate(calc1):
    agi1 = getAGI(calc1) + 0.001
    avgRate1 = np.asarray(calc1.records._combined) / agi1
    avgRate2 = np.where(avgRate1 > 1, 1.0, avgRate1)
    avgRate3 = np.where(avgRate2 < 0, 0., avgRate2)
    return avgRate3

def group_avgRate(calc, bins):
    agi = getAGI(calc)
    rate = getAvgRate(calc)
    wgt = calc.records.s006
    assert bins >= 1 and type(bins)==int
    size = int(219814 / bins)
    
    all_together = zip(agi, rate, wgt)
    all_together.sort()
    all_apart = zip(*all_together)
    agi_sorted = np.asarray(all_apart[0])
    rate_sorted = np.asarray(all_apart[1])
    wgt_sorted = np.asarray(all_apart[2])
    
    avgList = [0] * bins
    wgtList = [0] * bins
    
    for i in range(bins):
        #avgList[i] = (rate_sorted[(i*size):((i+1)*size)] * wgt_sorted[(i*size):((i+1)*size)]).sum()
        #wgtList[i] =  wgt_sorted[(i*size):((i+1)*size)].sum()
        #avgList[i] = np.percentile(rate_sorted[(i*size):((i+1)*size)], 75)
        avgList[i] = np.std(rate_sorted[(i*size):((i+1)*size)])
        wgtList[i] = 1
    
    avgFinal = np.asarray(avgList) / np.asarray(wgtList)
    return avgFinal

def group_avgRate_v2(calc, bins):
    agi = getAGI(calc)
    tax = calc.records._combined
    wgt = calc.records.s006
    ided_ind = np.where(calc_cl.records.s006 != 99999999999, True, False)
    #ided_ind = (calc_cl.records.c04470 > calc_cl.records._standard)
    assert bins >= 1 and type(bins)==int
    size = int(len(tax) / bins)
    
    all_together = zip(agi, tax, wgt, ided_ind)
    all_together.sort()
    all_apart = zip(*all_together)
    agi_sorted = np.asarray(all_apart[0])
    tax_sorted = np.asarray(all_apart[1])
    wgt_sorted = np.asarray(all_apart[2])
    ided_ind_sorted = np.asarray(all_apart[3])
    
    taxList = [0] * bins
    agiList = [0] * bins
    
    for i in range(bins):
        taxList[i] = np.where(ided_ind_sorted[(i*size):((i+1)*size)], tax_sorted[(i*size):((i+1)*size)] * wgt_sorted[(i*size):((i+1)*size)], 0).sum()
        agiList[i] = np.where(ided_ind_sorted[(i*size):((i+1)*size)], agi_sorted[(i*size):((i+1)*size)] * wgt_sorted[(i*size):((i+1)*size)], 0).sum()
        #taxList[i] = np.where(ided_ind_sorted[(i*size):((i+1)*size)], wgt_sorted[(i*size):((i+1)*size)], 0).sum()
        #agiList[i] = 1
        
    avgFinal = np.asarray(taxList) / np.asarray(agiList)
    return avgFinal

In [25]:
xx1 = group_avgRate(calc_cl, 5)
for x in xx1:
    print x
print '\n'
yy1 = group_avgRate(calc_all_beh, 5)
for y in yy1:
    print y
print '\n'
ww1 = group_avgRate(calc_bush_beh, 5)
for z in ww1:
    print z

0.170655433777
0.0548999661829
0.0608864508001
0.0744311651164
0.0802289930955


0.204166002523
0.044309357578
0.0479182629841
0.0546505735646
0.0527439738519


0.170348165444
0.0550909517804
0.0608891658046
0.0740903550653
0.0765334693071


In [14]:
xx2 = group_avgRate_v2(calc_cl, 5)
for x in xx2:
    print x
print '\n'
yy2 = group_avgRate_v2(calc_all_beh, 5)
for y in yy2:
    print y
print '\n'
ww2 = group_avgRate_v2(calc_bush_beh, 5)
for z in ww2:
    print z

54402649.8178
42584015.2284
38476798.3088
32319484.9119
4617041.5428


54011404.2899
42831848.6006
38271905.5822
32442471.886
4842359.45105




NameError: name 'calc_bush_beh' is not defined

## Advance calculators to 2026, recalculate average rates

In [11]:
calc_cl_adv = copy.deepcopy(calc_cl)
calc_bush_adv = copy.deepcopy(calc_bush)
calc_all_adv = copy.deepcopy(calc_all)
for i in range(9):
    calc_cl_adv.increment_year()
    calc_bush_adv.increment_year()
    calc_all_adv.increment_year()
assert calc_cl_adv.current_year == 2026
assert calc_bush_adv.current_year == 2026
assert calc_all_adv.current_year == 2026
calc_cl_adv.calc_all()
calc_bush_adv.calc_all()
calc_all_adv.calc_all()
calc_bush_adv_beh = Behavior.response(calc_cl_adv, calc_bush_adv)
calc_all_adv_beh = Behavior.response(calc_cl_adv, calc_all_adv)
print "Done"

Done


In [16]:
xx3 = group_avgRate(calc_cl_adv, 5)
for x in xx3:
    print x
print '\n'
yy3 = group_avgRate(calc_all_adv_beh, 5)
for y in yy3:
    print y
print '\n'
ww3 = group_avgRate(calc_bush_adv_beh, 5)
for z in ww3:
    print z

0.27527807894
0.106997221491
0.0890136000387
0.0918924585287
0.0842191643614


0.298470793555
0.100688460277
0.0781618278006
0.0753323592867
0.0618929170907


0.275166079786
0.10714679842
0.0890930545446
0.0914609976901
0.0886631710347


In [17]:
xx4 = group_avgRate_v2(calc_cl_adv, 5)
for x in xx4:
    print x
print '\n'
yy4 = group_avgRate_v2(calc_all_adv_beh, 5)
for y in yy4:
    print y
print '\n'
ww4 = group_avgRate_v2(calc_bush_adv_beh, 5)
for z in ww4:
    print z

0.0767977098015
0.131503859339
0.204552064376
0.257377491173
0.318178168698


0.0729914608927
0.131755081888
0.205901971434
0.269202696469
0.310970262779


0.0761971755516
0.131529450363
0.204556515907
0.257536243003
0.282028951461
